# Training a model on CarbonTracker's carbon flux
This notebook outlines the entire workflow to load and preprocess the following data sets, to be able to train a ML model:

- CarbonTracker
- ERA5 (monthly)
- ERA5-land (monthly)
- SPEI (monthly)
- MODIS (monthly)
- Biomass (yearly)
- Copernicus Landcover (yearly)

First follow the data downloading and config setup instuctions.

If you run this notebook on Surf Research Cloud, you shouldn't need to do this anymore.

In [1]:
import datetime
from pathlib import Path

import xarray as xr
from dask.distributed import Client

import excited_workflow
from excited_workflow import carbon_tracker
from excited_workflow.source_datasets import datasets


client = Client()

Define the paths for the carbon tracker dataset, the regions dataset and create output directory, as well as define the datasets that need to be included, the input variables (x_keys) for the model and the target variable (y_key).

In [2]:
cb_file= Path("/data/volume_2/EXCITED_prepped_data/CT2022.flux1x1-monthly.nc")
regions_file = Path("/data/volume_2/EXCITED_prepped_data/regions.nc")
output_path = Path.home()

time = datetime.datetime.now().strftime("%Y-%m-%d_%H_%M")
output_dir = output_path / f"carbon_tracker-{time}"
output_dir.mkdir(parents=True, exist_ok=True)

desired_data = [
    "biomass",
    "spei",
    "modis",
    "era5_monthly",
    "era5_land_monthly",
    "copernicus_landcover"
]

x_keys = ["d2m", "mslhf", "msshf", "ssr", "str", "t2m", "spei", "NIRv", "skt",
            "stl1", "swvl1", "lccs_class"]
y_key = "bio_flux_opt"

Write input variables to a text file, so that they can be read in when running the model.

In [3]:
with open(output_dir / "model_variables.txt", "w") as f:
    f.write(repr(x_keys) + "\n")
    f.write(repr(y_key) + "\n")
    f.write(repr(desired_data) + "\n")  

Merge the desired datasets into a single xr.Dataset with the same dimensions as the carbon tracker dataset. 

In [4]:
ds_cb = xr.open_dataset(cb_file)
ds_cb = excited_workflow.utils.convert_timestamps(ds_cb)
ds_input = xr.merge(
    [
        datasets[name].load(freq="monthly", target_grid=ds_cb)
        for name in desired_data
    ]
)

To limit the analyis to Transcom region 2 (North America) we require the `regions.nc` file:

In [5]:
ds_na, df_na = carbon_tracker.mask_region(regions_file, cb_file, ds_input, 2, y_key)

In [6]:

ds_na


<xarray.Dataset>
Dimensions:           (time: 240, latitude: 180, longitude: 360)
Coordinates:
  * longitude         (longitude) float64 -179.5 -178.5 -177.5 ... 178.5 179.5
  * latitude          (latitude) float64 -89.5 -88.5 -87.5 ... 87.5 88.5 89.5
  * time              (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2019-12-01
Data variables: (12/26)
    bio_flux_opt      (time, latitude, longitude) float64 nan nan ... nan nan
    transcom_regions  (latitude, longitude) float64 nan nan nan ... nan nan nan
    biomass           (time, latitude, longitude) float64 nan nan ... nan nan
    spei              (time, latitude, longitude) float64 nan nan ... nan nan
    NDVI              (time, latitude, longitude) float64 nan nan ... nan nan
    NIRv              (time, latitude, longitude) float64 nan nan ... nan nan
    ...                ...
    stl4              (time, latitude, longitude) float64 nan nan ... nan nan
    swvl1             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl2             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl3             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl4             (time, latitude, longitude) float64 nan nan ... nan nan
    lccs_class        (time, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    averaging_period_length_hours:  744
    email:                          carbontracker.team@noaa.gov
    url:                            http://carbontracker.noaa.gov
    institution:                    NOAA Earth System Research Laboratory
    Conventions:                    CF-1.5
    history:                        Time-stamp: <Orion-login-1.HPC.MsState.Ed...
    NCO:                            netCDF Operators version 4.9.3 (Homepage ...
    version:                        CT2022 1x1 3-hourly fluxes as of 2023-01-...

Validate the model by splitting the dataset into 5 groups. Train the model over 4 groups and predict over the remaining group iteratively. Output rmse netcdfs and scatterplots are stored in the output directory.

In [7]:
carbon_tracker.validate_model(ds_na, 5, x_keys, y_key, output_dir)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.77790,0.00000,3.75600,1.20000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76280,0.00000,4.29880,1.17000


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76580,0.00000,3.72830,1.17000


2024-01-15 13:00:53,115 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-01-15 13:00:53,593 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76380,0.00000,4.94940,1.16000


...

2024-01-15 13:01:02,194 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-01-15 13:01:02,727 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-01-15 13:01:03,208 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.75930,0.00000,8.02940,1.16000


2024-01-15 13:01:07,994 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-01-15 13:01:08,337 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-01-15 13:01:08,833 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)
2024-01-15 13:01:09,558 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-01-15 13:01:09,789 - distributed.utils_perf - WARNING - full garbage collections took 23% CPU time recently (threshold: 10%)
2024-01-15 13:01:11,294 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)


2024-01-15 13:01:11,687 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
2024-01-15 13:01:12,208 - distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)


<Figure size 600x600 with 0 Axes>

Train the model over the entire dataset.

In [8]:
pycs, model = carbon_tracker.train_model(df_na, x_keys, y_key)

2024-01-15 13:01:15,012 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-01-15 13:01:17,904 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.00000,0.00000,0.00000,0.76450,0.00000,4.07700,1.45000


2024-01-15 13:01:20,668 - distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
2024-01-15 13:01:21,094 - distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
2024-01-15 13:01:21,685 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-01-15 13:01:22,519 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-01-15 13:01:22,757 - distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
2024-01-15 13:01:24,580 - distributed.utils_perf - WARNING - full garbage collections took 20% CPU time recently (threshold: 10%)


Save the model to ONNX in the output directory. 

In [9]:

carbon_tracker.save_model(pycs, model, output_dir)

The maximum opset needed by this model is only 8.


Create dataframe to run the model with. 

In [10]:
df = df_na[x_keys]
df.head()

d2m       mslhf      msshf         ssr  \
time       latitude longitude                                                  
2000-08-01 16.5     -97.5      291.160736 -123.552444 -29.256828  17632392.0   
                    -96.5      287.451385  -98.636139 -44.350342  17875046.0   
                    -95.5      292.602661  -88.933502 -76.770813  18379498.0   
           17.5     -100.5     290.568329 -118.364532 -51.205177  17707054.0   
                    -99.5      290.078156 -119.800644 -42.374344  17862124.0   

                                     str         t2m      spei      NIRv  \
time       latitude longitude                                              
2000-08-01 16.5     -97.5     -4539167.0  294.455078  0.402637  0.264352   
                    -96.5     -5688396.0  291.743347  0.859432  0.186314   
                    -95.5     -4277693.5  298.020874 -0.002091  0.245570   
           17.5     -100.5    -3109625.0  293.402252  0.364373  0.270905   
                    -99.5     -3914028.0  293.219940  0.007593  0.249498   

                                      skt        stl1     swvl1  lccs_class  
time       latitude longitude                                                
2000-08-01 16.5     -97.5      293.800140  294.488708  0.405656        90.0  
                    -96.5      291.346344  292.329987  0.441099       120.0  
                    -95.5      298.976379  299.503906  0.340303        60.0  
           17.5     -100.5     293.096253  293.790924  0.412808        90.0  
                    -99.5      291.973236  292.737579  0.406332       120.0

Open model and run it over the dataframe to check it was saved correctly. 

In [11]:
from onnxruntime import InferenceSession


with open(output_dir / "lightgbm.onnx", "rb") as f:
    model = f.read()

sess = InferenceSession(model)
predictions_onnx = sess.run(None, {'X': df.to_numpy()})[0]

In [12]:
predictions_onnx

array([[-1.1125659e-06],
       [-6.6243564e-07],
       [-5.5546332e-07],
       ...,
       [ 4.6423793e-07],
       [ 4.8798864e-07],
       [ 1.2227741e-07]], dtype=float32)